In [1]:
import os
import numpy as np
#from proxy_env import RLBot
import matplotlib.pyplot as plt
import pickle
from keras.models import load_model, Sequential
from keras.layers import Dense, Activation
from keras.utils.vis_utils import plot_model
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

%matplotlib inline

/home/daniel/anaconda2/envs/tensorflow/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
observations = {}
observations['proxy_sensor'] = [0.1,0.2,0.3,0.4,3,4,5,6,0.001,0,0,1,0.001,4,5,0]
observations['light_sensor'] = []
observations['proxy_sensor'] = np.asarray(observations['proxy_sensor'])
observations['proxy_sensor'][observations['proxy_sensor'] < 0.001] = 2
print observations['proxy_sensor'].shape


(16,)


In [3]:
reward_robot = {}
reward_robot['proxy_sensor'] = (observations['proxy_sensor'] < 0.7).sum() * -2
reward_robot['proxy_sensor'] += (observations['proxy_sensor'] == 0).sum() * -10
print reward_robot
r = np.clip(np.sum(np.absolute([0.8,0.8])) * 2, 0, 2)
reward_robot['proxy_sensor'] += r
print reward_robot

{'proxy_sensor': -12}
{'proxy_sensor': -10.0}


In [5]:
def train():
    #Initializing model
    print 'Building Model.'
    
    
    model = load_model('model_proxy_conf2_small_mazze_4actions.hdf5')
        
    print 'Saved model found.'

    
            
    #Initializing environment
    #env = RLBot()

    # Set learning parameters
    y = .99
    e = 0.1
    num_epochs = 10
    num_steps = 300
    # create lists to contain total rewards and steps per episode
    stepList = []
    rewardList = []
    lossList = []
    SPEED = 0.8

    for i in range(num_epochs):
        # Reset environment and get first new observation
        #env.reset()
        state, reward = observations, reward_robot
        
        state = state['proxy_sensor'].reshape((1, -1))
        
        reward = reward['proxy_sensor']
        Q = model.predict(state)
        action = Q.argmax()
        rAll = 0
        done = False
        loss = 0
        # The Q-Network"""
        for j in range(num_steps):
            print("Step {} | Action: {} | Reward: {}".format(j, action, reward))
            # Choose an action by greedily (with e chance of random action)
            # from the Q-network
            Q = model.predict(state)
            print Q
            action = Q.argmax()
           
            # random action
            if np.random.rand(1) < e:
                action = 5
                print("e = {}. Choosing Random Action: {}".format(e, action))
            # Get new state and reward from environment
            speed = np.zeros(2)
            # Q -> left, forward, backward, break
            if action == 0:
                speed[0] = 0
                speed[1] = SPEED
            if action == 1:
                speed[0] = -SPEED
                speed[1] = -SPEED
            if action == 2:
                speed[0] = SPEED
                speed[1] = SPEED
            if action == 3:
                speed[0] = 0
                speed[1] = 0
            if action == 4:
                speed[0] = SPEED
                speed[1] = 0

            state_, reward_ = observations, reward_robot
            state_ = state_['proxy_sensor'].reshape((1, -1))
            reward_ = reward_['proxy_sensor']
            # Obtain the Q' values by feeding the new state through our network
            Q_ = model.predict(state_)
            # Obtain maxQ' and set our target value for chosen action.
            maxQ_ = np.max(Q_)
            targetQ = Q
            
            action = action-1
            
            targetQ[0, action] = reward + y * maxQ_
            print targetQ.shape
            # Train our network using target and predicted Q values
            loss += model.train_on_batch(state, targetQ)
            rAll += reward
            state = state_
            reward = reward_
            if done is True:
                break
                
        # Reduce chance of random action as we train the model.
        if num_epochs <= 5:
            e -= 0.01 #0.01
            
        else:
            e -= 0.001 #0.002
        stepList.append(j)
        rewardList.append(rAll)
        lossList.append(loss)
        print("Episode: " + str(i))
        print("Loss: " + str(loss))
        print("e: " + str(e))
        print("Reward: " + str(rAll))
        pickle.dump({'stepList': stepList, 'rewardList': rewardList, 'lossList': lossList},
                    open("history_porxy_conf2.p", "wb"))
        model.save('model_proxy_conf2.hdf5')
        
    print("Average loss: " + str(sum(lossList) / num_episodes))
    print("Average number of steps: " + str(sum(stepList) / num_episodes))
    print("Average reward: " + str(sum(rewardList) / num_episodes))

    plt.plot(rewardList)
    plt.plot(stepList)
    plt.plot(lossList)
    return i

if __name__ == '__main__':
    try:
        train()
    except KeyboardInterrupt or i == 100:
        print('Exiting.')


Building Model.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Saved model found.
Step 0 | Action: 4 | Reward: -10.0
[[12.85325  12.897898  0.       12.851906 12.946903]]
(1, 5)
Step 1 | Action: 3 | Reward: -10.0
[[12.871836 12.942874  0.       12.875218 12.968123]]
(1, 5)
Step 2 | Action: 3 | Reward: -10.0
[[12.8950205 12.983679   0.        12.881611  12.992161 ]]
(1, 5)
Step 3 | Action: 3 | Reward: -10.0
[[12.922458 13.020801  0.       12.872815 13.018841]]
(1, 5)
Step 4 | Action: 0 | Reward: -10.0
[[12.960274 13.087246  0.       12.898879 13.070956]]
(1, 5)
Step 5 | Action: 0 | Reward: -10.0
[[13.007175 13.180065  0.       12.956138 13.145856]]
(1, 5)
Step 6 | Action: 0 | Reward: -10.0
[[13.061884 13.296603  0.       13.041355 13.241137]]
(1, 5)
Step 7 | Action: 0 | Reward: -10.0
[[13.123182 13.434478  0.       13.151666 13.354631]]
(1, 5)
Step 8 | Action: 0 | Reward: -10.0
[[13.189874

(1, 5)
Step 107 | Action: 2 | Reward: -10.0
[[ 9.282856 25.037004  0.       25.835205 16.982647]]
(1, 5)
Step 108 | Action: 2 | Reward: -10.0
[[ 9.154091 25.023966  0.       25.843443 16.867826]]
(1, 5)
Step 109 | Action: 2 | Reward: -10.0
[[ 9.055736 25.020927  0.       25.857302 16.783455]]
(1, 5)
Step 110 | Action: 2 | Reward: -10.0
[[ 8.998342 25.033115  0.       25.880623 16.741098]]
(1, 5)
Step 111 | Action: 2 | Reward: -10.0
[[ 8.946509 25.044365  0.       25.90198  16.702858]]
(1, 5)
Step 112 | Action: 2 | Reward: -10.0
[[ 8.881371 25.04618   0.       25.915432 16.648611]]
(1, 5)
Step 113 | Action: 2 | Reward: -10.0
[[ 8.822514 25.047953  0.       25.927748 16.599588]]
(1, 5)
Step 114 | Action: 2 | Reward: -10.0
[[ 8.76428  25.047297  0.       25.937319 16.549847]]
(1, 5)
Step 115 | Action: 2 | Reward: -10.0
[[ 8.708658 25.04537   0.       25.945055 16.501656]]
(1, 5)
Step 116 | Action: 2 | Reward: -10.0
[[ 8.658423 25.04368   0.       25.95212  16.458122]]
(1, 5)
Step 117 | Ac

Step 217 | Action: 2 | Reward: -10.0
[[ 7.8263636 24.923065   0.        25.9628    15.713716 ]]
(1, 5)
Step 218 | Action: 2 | Reward: -10.0
[[ 7.825876 24.922928  0.       25.962732 15.713259]]
(1, 5)
Step 219 | Action: 2 | Reward: -10.0
[[ 7.825443 24.922812  0.       25.962677 15.712855]]
(1, 5)
Step 220 | Action: 2 | Reward: -10.0
[[ 7.825047 24.922703  0.       25.962624 15.712486]]
(1, 5)
Step 221 | Action: 2 | Reward: -10.0
[[ 7.8246922 24.92261    0.        25.962578  15.712154 ]]
(1, 5)
Step 222 | Action: 2 | Reward: -10.0
[[ 7.824374 24.922523  0.       25.96254  15.711861]]
(1, 5)
Step 223 | Action: 2 | Reward: -10.0
[[ 7.824081 24.922438  0.       25.962494 15.711584]]
e = 0.1. Choosing Random Action: 5
(1, 5)
Step 224 | Action: 4 | Reward: -10.0
[[ 7.8230214 24.922144   0.        25.962349  15.710595 ]]
(1, 5)
Step 225 | Action: 2 | Reward: -10.0
[[ 7.822065 24.921875  0.       25.962212 15.709702]]
(1, 5)
Step 226 | Action: 2 | Reward: -10.0
[[ 7.821204 24.921629  0.      

(1, 5)
Step 9 | Action: 2 | Reward: -10.0
[[ 7.8128366 24.91901    0.        25.960613  15.700993 ]]
(1, 5)
Step 10 | Action: 2 | Reward: -10.0
[[ 7.8128247 24.91901    0.        25.960615  15.700982 ]]
(1, 5)
Step 11 | Action: 2 | Reward: -10.0
[[ 7.812816 24.919006  0.       25.960613 15.700974]]
e = 0.099. Choosing Random Action: 5
(1, 5)
Step 12 | Action: 4 | Reward: -10.0
[[ 7.812812 24.919006  0.       25.960615 15.700972]]
e = 0.099. Choosing Random Action: 5
(1, 5)
Step 13 | Action: 4 | Reward: -10.0
[[ 7.8128104 24.919008   0.        25.960613  15.70097  ]]
(1, 5)
Step 14 | Action: 2 | Reward: -10.0
[[ 7.812809 24.919008  0.       25.960615 15.700969]]
(1, 5)
Step 15 | Action: 2 | Reward: -10.0
[[ 7.812807 24.919006  0.       25.960615 15.700969]]
(1, 5)
Step 16 | Action: 2 | Reward: -10.0
[[ 7.812806 24.919008  0.       25.960617 15.700967]]
(1, 5)
Step 17 | Action: 2 | Reward: -10.0
[[ 7.8128047 24.919006   0.        25.960615  15.700966 ]]
(1, 5)
Step 18 | Action: 2 | Rewar

Step 100 | Action: 2 | Reward: -10.0
[[ 7.812958 24.919151  0.       25.960752 15.701136]]
(1, 5)
Step 101 | Action: 2 | Reward: -10.0
[[ 7.8129535 24.919147   0.        25.960749  15.701134 ]]
(1, 5)
Step 102 | Action: 2 | Reward: -10.0
[[ 7.8129463 24.91914    0.        25.960741  15.701125 ]]
(1, 5)
Step 103 | Action: 2 | Reward: -10.0
[[ 7.812945 24.919138  0.       25.960741 15.701122]]
(1, 5)
Step 104 | Action: 2 | Reward: -10.0
[[ 7.812945 24.919138  0.       25.960741 15.701122]]
(1, 5)
Step 105 | Action: 2 | Reward: -10.0
[[ 7.8129435 24.919142   0.        25.960743  15.701122 ]]
(1, 5)
Step 106 | Action: 2 | Reward: -10.0
[[ 7.8129444 24.919146   0.        25.960749  15.701125 ]]
(1, 5)
Step 107 | Action: 2 | Reward: -10.0
[[ 7.812951 24.919159  0.       25.960764 15.701137]]
(1, 5)
Step 108 | Action: 2 | Reward: -10.0
[[ 7.812954 24.919167  0.       25.960773 15.701139]]
(1, 5)
Step 109 | Action: 2 | Reward: -10.0
[[ 7.812962 24.919182  0.       25.960787 15.701151]]
(1, 5)


...

Step 255 | Action: 2 | Reward: -10.0
[[ 7.8220344 24.930326   0.        25.971827  15.712092 ]]
(1, 5)
Step 256 | Action: 2 | Reward: -10.0
[[ 7.822048 24.930347  0.       25.971846 15.712108]]
(1, 5)
Step 257 | Action: 2 | Reward: -10.0
[[ 7.8220577 24.930365   0.        25.971867  15.712121 ]]
(1, 5)
Step 258 | Action: 2 | Reward: -10.0
[[ 7.8220673 24.93038    0.        25.97188   15.712131 ]]
(1, 5)
Step 259 | Action: 2 | Reward: -10.0
[[ 7.8220673 24.93038    0.        25.97188   15.712131 ]]
(1, 5)
Step 260 | Action: 2 | Reward: -10.0
[[ 7.8220553 24.930359   0.        25.97186   15.712116 ]]
(1, 5)
Step 261 | Action: 2 | Reward: -10.0
[[ 7.822036 24.930325  0.       25.97182  15.71209 ]]
(1, 5)
Step 262 | Action: 2 | Reward: -10.0
[[ 7.8220177 24.930294   0.        25.971794  15.712064 ]]
(1, 5)
Step 263 | Action: 2 | Reward: -10.0
[[ 7.822001 24.930265  0.       25.971766 15.712042]]
(1, 5)
Step 264 | Action: 2 | Reward: -10.0
[[ 7.821986 24.930239  0.       25.971739 15.712022

(1, 5)
Step 59 | Action: 2 | Reward: -10.0
[[ 7.822253 24.930504  0.       25.971992 15.712341]]
(1, 5)
Step 60 | Action: 2 | Reward: -10.0
[[ 7.8222313 24.930466   0.        25.97195   15.71231  ]]
(1, 5)
Step 61 | Action: 2 | Reward: -10.0
[[ 7.822217 24.930439  0.       25.971926 15.712289]]
(1, 5)
Step 62 | Action: 2 | Reward: -10.0
[[ 7.8222   24.93041   0.       25.971897 15.712266]]
(1, 5)
Step 63 | Action: 2 | Reward: -10.0
[[ 7.822192  24.930397   0.        25.97188   15.7122555]]
(1, 5)
Step 64 | Action: 2 | Reward: -10.0
[[ 7.8221917 24.930391   0.        25.971876  15.712255 ]]
(1, 5)
Step 65 | Action: 2 | Reward: -10.0
[[ 7.8221955 24.930393   0.        25.97188   15.712256 ]]
(1, 5)
Step 66 | Action: 2 | Reward: -10.0
[[ 7.822202 24.930407  0.       25.971888 15.712265]]
(1, 5)
Step 67 | Action: 2 | Reward: -10.0
[[ 7.822199 24.930399  0.       25.971882 15.712261]]
(1, 5)
Step 68 | Action: 2 | Reward: -10.0
[[ 7.8221846 24.930374   0.        25.971855  15.71224  ]]
(1, 5

[[ 7.821831 24.929846  0.       25.97133  15.711776]]
(1, 5)
Step 166 | Action: 2 | Reward: -10.0
[[ 7.821763 24.929834  0.       25.971329 15.711716]]
(1, 5)
Step 167 | Action: 2 | Reward: -10.0
[[ 7.8217134 24.929842   0.        25.971346  15.711676 ]]
(1, 5)
Step 168 | Action: 2 | Reward: -10.0
[[ 7.8216734 24.929857   0.        25.971369  15.711645 ]]
(1, 5)
Step 169 | Action: 2 | Reward: -10.0
[[ 7.821644 24.929882  0.       25.971403 15.711629]]
(1, 5)
Step 170 | Action: 2 | Reward: -10.0
[[ 7.8216257 24.929918   0.        25.971449  15.711625 ]]
(1, 5)
Step 171 | Action: 2 | Reward: -10.0
[[ 7.821599 24.929935  0.       25.971472 15.711607]]
(1, 5)
Step 172 | Action: 2 | Reward: -10.0
[[ 7.8215823 24.929962   0.        25.971504  15.711601 ]]
(1, 5)
Step 173 | Action: 2 | Reward: -10.0
[[ 7.821572 24.929989  0.       25.971539 15.7116  ]]
(1, 5)
Step 174 | Action: 2 | Reward: -10.0
[[ 7.82156  24.930014  0.       25.971567 15.711599]]
(1, 5)
Step 175 | Action: 2 | Reward: -10.0


[[ 7.824009 24.932793  0.       25.974281 15.714492]]
(1, 5)
Step 278 | Action: 2 | Reward: -10.0
[[ 7.824007 24.9328    0.       25.974293 15.714494]]
(1, 5)
Step 279 | Action: 2 | Reward: -10.0
[[ 7.8240104 24.93281    0.        25.974308  15.714501 ]]
(1, 5)
Step 280 | Action: 2 | Reward: -10.0
[[ 7.8240194 24.932835   0.        25.974329  15.714515 ]]
(1, 5)
Step 281 | Action: 2 | Reward: -10.0
[[ 7.824025 24.932854  0.       25.974352 15.714527]]
(1, 5)
Step 282 | Action: 2 | Reward: -10.0
[[ 7.8240232 24.93286    0.        25.974356  15.714527 ]]
(1, 5)
Step 283 | Action: 2 | Reward: -10.0
[[ 7.8240337 24.932882   0.        25.974384  15.714543 ]]
(1, 5)
Step 284 | Action: 2 | Reward: -10.0
[[ 7.8240385 24.932896   0.        25.974398  15.714552 ]]
(1, 5)
Step 285 | Action: 2 | Reward: -10.0
[[ 7.824049 24.93292   0.       25.974424 15.714569]]
(1, 5)
Step 286 | Action: 2 | Reward: -10.0
[[ 7.82406  24.932938  0.       25.974443 15.714582]]
(1, 5)
Step 287 | Action: 2 | Reward: -

NameError: global name 'num_episodes' is not defined